# Exploring Hacker News Posts

In this project, we will compare two different types of posts from Hacker News, a site where technology related stories (or 'posts') are voted and commented upon. The two types of posts begin with either 'Ask HN' or 'Show HN'.

Users submit 'Ask HN' posts to ask the Hacker News community a specific question, such as "What is the best online course you've ever taken?" Likewise, users submit 'Show HN' posts to show the Hacker News community a project, product, or generally something interesting.

We will compare these two types of posts to determine:

- Do 'Ask HN' or 'Show HN' receive more comments on average?
- Do posts created at a certain time receive more comments on average?

The data set was reduced from 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

## Introduction

We will first read in the data and remove the headers

In [1]:
# read in the data
import csv

f = open(r'C:\projectdatasets\hacker_news.csv', encoding="utf8")
hn = list(csv.reader(f))

# show the first five rows
hn[:5]

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

In [3]:
# remove the headers
headers = hn[0]
hn = hn[1:]

# print the header row
print(headers)
print()

# print the first five rows
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']

[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


## Extracting 'Ask HN' and 'Show HN' Posts

We will identify posts that begin with either 'Ask HN' or 'Show HN' and separate the data for those two types of posts into different lists, making it easier to analyse

In [4]:
# import regular expressions package for string matching
import re
ask_posts = []
show_posts = []
other_posts = []

for post in hn:
    title = post[1]
    
    # find string matches and set each to a boolean True or False
    match1 = re.search(r"^Ask HN", title, re.I)
    match2 = re.search(r"^Show HN", title, re.I)
    
    if match1:
        ask_posts.append(post)
    elif match2:
        show_posts.append(post)
    else:
        other_posts.append(post)

# show the lengths of each list
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


## Calculating the Average Number of Comments

We will calculate the average number of comments each type of post receives

In [8]:
# for 'Ask HN'
ask_comments = [int(post[4]) for post in ask_posts]
avg_ask_comments = sum(ask_comments) / len(ask_posts)
print(avg_ask_comments)

14.038417431192661


In [10]:
# for 'Show HN'
show_comments = [int(post[4]) for post in show_posts]
avg_show_comments = sum(show_comments) / len(show_posts)
print(avg_show_comments)

10.31669535283993


On average, 'Ask HN' posts receive approximately 14 comments, whereas 'Show HN' posts receive approximately 10. Since 'Ask HN' posts are more likely to receive comments, we will focus our remaining analysis just on these posts

## Finding the Amount of 'Ask HN' Posts and Comments by Hour Created

We will determine if we can maximize the amount of comments an 'Ask HN' post receives by creating it at a certain time. 

First, we will find the amount of 'Ask HN' posts created during each hour of day, with the number of comments those posts received. 

Then, we will calculate the average amount of comments 'Ask HN' posts created at each hour of the day receive.

In [35]:
# calculate the amount of `Ask HN` posts created during each hour of day 
# and the number of comments received

import datetime as dt

# create a list of numbers of comments, for each post
ask_comments = [int(post[4]) for post in ask_posts]

# create list of dates, for each post
created_date = [post[6] for post in ask_posts]

# create a 'ZIP' object
result_list = zip(created_date, ask_comments)

In [36]:
# show the raw date format
created_date[0:5]

['8/16/2016 9:55',
 '11/22/2015 13:43',
 '5/2/2016 10:14',
 '8/2/2016 14:20',
 '10/15/2015 16:38']

In [37]:
comments_by_hour = {}
counts_by_hour = {}
date_format = "%m/%d/%Y %H:%M"

for date, comment in result_list:
    time = dt.datetime.strptime(date, date_format).strftime("%H")
    if time in counts_by_hour:
        comments_by_hour[time] += comment
        counts_by_hour[time] += 1
    else:
        comments_by_hour[time] = comment
        counts_by_hour[time] = 1

# show total aggregate of comments per hour
comments_by_hour

{'09': 251,
 '13': 1253,
 '10': 793,
 '14': 1416,
 '16': 1814,
 '23': 543,
 '12': 687,
 '17': 1146,
 '15': 4477,
 '21': 1745,
 '20': 1722,
 '02': 1381,
 '18': 1439,
 '03': 421,
 '05': 464,
 '19': 1188,
 '01': 683,
 '22': 479,
 '08': 492,
 '04': 337,
 '00': 447,
 '06': 397,
 '07': 267,
 '11': 641}

In [38]:
# show total count of posts per hour
counts_by_hour

{'09': 45,
 '13': 85,
 '10': 59,
 '14': 107,
 '16': 108,
 '23': 68,
 '12': 73,
 '17': 100,
 '15': 116,
 '21': 109,
 '20': 80,
 '02': 58,
 '18': 109,
 '03': 54,
 '05': 46,
 '19': 110,
 '01': 60,
 '22': 71,
 '08': 48,
 '04': 47,
 '00': 55,
 '06': 44,
 '07': 34,
 '11': 58}

In [50]:
# calculate the average amount of comments 'Ask HN' posts created at each hour
avg_by_hour = [[hr, comments_by_hour[hr] / counts_by_hour[hr]] for hr in comments_by_hour]
avg_by_hour

[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

## Sorting and Printing Values

In [56]:
# sort the values
avg_by_hour = sorted(avg_by_hour, key=lambda x: x[1], reverse=True)
avg_by_hour

[['15', 38.5948275862069],
 ['02', 23.810344827586206],
 ['20', 21.525],
 ['16', 16.796296296296298],
 ['21', 16.009174311926607],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['18', 13.20183486238532],
 ['17', 11.46],
 ['01', 11.383333333333333],
 ['11', 11.051724137931034],
 ['19', 10.8],
 ['08', 10.25],
 ['05', 10.08695652173913],
 ['12', 9.41095890410959],
 ['06', 9.022727272727273],
 ['00', 8.127272727272727],
 ['23', 7.985294117647059],
 ['07', 7.852941176470588],
 ['03', 7.796296296296297],
 ['04', 7.170212765957447],
 ['22', 6.746478873239437],
 ['09', 5.5777777777777775]]

In [63]:
# print the first 5 hours with the highest average comments
print("Top 5 Hours for 'Ask HN' Comments")
for hr, avg in avg_by_hour[:5]:
    print("{}: {:.2f} average comments per post".format(dt.datetime.strptime(hr, "%H").strftime("%H:%M"),avg))

Top 5 Hours for 'Ask HN' Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


The hour that receives the most comments per post on average is 15:00 (3pm), with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

## Conclusion

In this project, we analyzed 'Ask HN' and 'Show HN' posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we recommend the post be categorised as 'Ask HN', and created between 3:00-4:00pm

However, the data set we analysed excluded posts without any comments. So it's more accurate to say that of the posts that received comments, 'Ask HN' posts received more comments on average, and 'Ask HN' posts created between 3:00-4:00pm) received the most comments on average.